In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.preprocessing import StandardScaler
import pyarrow.parquet as pq
import pyarrow as pa
import operator

In [ ]:
df_model = pd.read_parquet('data/recommendation_model.parquet')

In [ ]:
df_model

In [ ]:
# se crea la tabla pivot
piv_table = df_model.pivot_table(index=['user_id'], columns=['item_name'], values='combined_metric')
piv_table

In [ ]:
# Se normaliza la matriz pivot con la formula de normalizacion
piv_table_norm = piv_table.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

In [ ]:
# Se transpone la matriz, para tener juegos en filas
piv_table_norm.fillna(0, inplace=True)
piv_table_norm = piv_table_norm.T
# Se manteniene solo las columnas que tienen al menos un valor distinto de cero
piv_table_norm = piv_table_norm.loc[:, (piv_table_norm != 0).any(axis=0)]

In [ ]:
piv_table_norm

In [ ]:
# Reemplazar los valores infinitos con un valor específico
piv_table_norm.replace([np.inf, -np.inf], np.nan, inplace=True)
piv_table_norm.fillna(0, inplace=True)

In [ ]:
# Se crea una tabla sparse que solo guarda valores distintos de cero y permite optimizar 
piv_sparse = sp.sparse.csr_matrix(piv_table_norm.values)
piv_sparse

In [ ]:
# Se deja en filas para cada uno de los endpoints el respectivo items o user segun corresponda, para hacer operaciones
item_simil = cosine_similarity(piv_sparse)
user_simil = cosine_similarity(piv_sparse.T)

In [ ]:
# Se ordenan en df para poder exportarlos luego y para manejarlos más sencillo
df_item_simil = pd.DataFrame(item_simil, index = piv_table_norm.index, columns = piv_table_norm.index)
df_user_simil = pd.DataFrame(user_simil, index = piv_table_norm.columns, columns = piv_table_norm.columns)

In [ ]:
df_item_simil.head(2)

In [ ]:
df_user_simil.head(2)

In [ ]:
#se hace una funcion para que tenga de entrada el juego y el dataframe y busque los juegos más similares
def recommended_games_item(game, df_item_simil):
    similar_games = {}
    count = 1
    for item in df_item_simil.sort_values(by=game, ascending=False).index[1:6]:
        similar_games[f"Recomendación {count}"] = item
        count += 1
    return similar_games

In [ ]:
recommended_games_item('0RBITALIS',df_item_simil)

In [ ]:
'''
    Se crea una funcion para las recomendaciones basadas en un usuario, para esto
    se tomarán las calificaciones que tienen los usuarios similares 
    y las veces que se recomienda cada juego por los usuarios similares.

'''
def similar_user_recs(user):
    
    # Se verifica si el usuario está presente en las columnas de piv_table_norm
    if user not in piv_table_norm.columns:
        return {'message': 'El Usuario no tiene datos disponibles {}'.format(user)}

    # Se obtienen los usuarios más similares 
    sim_users = df_user_simil.sort_values(by=user, ascending=False).index[1:11]

    best = []  
    most_common = {}  

    # Por cada usuario similar, encuentra el juego mejor calificado y lo agrega a la lista 'best'
    for i in sim_users:
        max_score = piv_table_norm.loc[:, i].max()
        best.append(piv_table_norm[piv_table_norm.loc[:, i] == max_score].index.tolist())

    # Se cuenta cuántas veces se recomienda cada juego
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1

    # Se ordenan los juegos de mayor recomendacion
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)

    return dict(sorted_list[:5])

In [ ]:
similar_user_recs('76561197970982479')

In [ ]:
pq.write_table(pa.Table.from_pandas(piv_table_norm), 'datos/piv_table_norm.parquet')
pq.write_table(pa.Table.from_pandas(df_user_simil), 'datos/df_user_simil.parquet')
pq.write_table(pa.Table.from_pandas(df_item_simil), 'datos/df_item_simil.parquet')